In [ ]:
# Layout module for PDF generation

from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm, cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph, Image
from reportlab.lib.enums import TA_CENTER, TA_RIGHT, TA_LEFT

# Company colors
PRIMARY_COLOR = colors.HexColor("#1A365D")  # Dark blue
SECONDARY_COLOR = colors.HexColor("#3182CE")  # Medium blue
ACCENT_COLOR = colors.HexColor("#63B3ED")  # Light blue
TEXT_COLOR = colors.HexColor("#2D3748")  # Dark gray
LIGHT_BG_COLOR = colors.HexColor("#EDF2F7")  # Very light gray/blue

def process_header_image(canvas_obj, header_image_path, width=A4[0], height=40*mm):
    """
    Places the company logo/header image at the top of the page

    Args:
        canvas_obj: The ReportLab canvas object
        header_image_path: Path to the header image file
        width: Width to which the image should be scaled
        height: Height to which the image should be scaled
    """
    try:
        canvas_obj.saveState()
        img = Image(header_image_path, width=width, height=height)
        img.drawOn(canvas_obj, 0, A4[1] - height)
        canvas_obj.restoreState()
    except Exception as e:
        # If image can't be loaded, create a placeholder header
        canvas_obj.saveState()
        canvas_obj.setFillColor(PRIMARY_COLOR)
        canvas_obj.rect(0, A4[1] - height, width, height, fill=1)
        canvas_obj.setFillColor(colors.white)
        canvas_obj.setFont("Helvetica-Bold", 16)
        canvas_obj.drawCentredString(A4[0]/2, A4[1] - height/2, "COMPANY NAME")
        canvas_obj.restoreState()

def add_footer(canvas_obj, contact_info="Email: info@example.com | Phone: (123) 456-7890 | www.example.com"):
    """
    Adds a footer with contact information to the PDF

    Args:
        canvas_obj: The ReportLab canvas object
        contact_info: Contact information text to display
    """
    footer_height = 15*mm

    # Footer background
    canvas_obj.saveState()
    canvas_obj.setFillColor(LIGHT_BG_COLOR)
    canvas_obj.rect(0, 0, A4[0], footer_height, fill=1)

    # Contact info
    canvas_obj.setFillColor(TEXT_COLOR)
    canvas_obj.setFont("Helvetica", 8)
    canvas_obj.drawCentredString(A4[0]/2, footer_height/2, contact_info)

    # Page number
    page_num = canvas_obj.getPageNumber()
    canvas_obj.drawRightString(A4[0] - 2*cm, footer_height/2, f"Page {page_num}")
    canvas_obj.restoreState()

def create_layout(output_path, title="Property Report", header_image_path=None):
    """
    Creates the basic PDF layout including background, header, title area, and footer

    Args:
        output_path: Path where PDF will be saved
        title: Title of the document
        header_image_path: Path to the header image file

    Returns:
        A tuple containing:
        - canvas object
        - content_start_y: Y-coordinate where content should start
        - content_width: Available width for content
        - content_height: Available height for content
    """
    # Initialize canvas
    c = canvas.Canvas(output_path, pagesize=A4)
    width, height = A4

    # Constants for layout
    margin = 2.5*cm
    header_height = 40*mm
    title_height = 20*mm
    footer_height = 15*mm

    # Add background color
    c.setFillColor(colors.white)
    c.rect(0, 0, width, height, fill=1)

    # Add header image
    if header_image_path:
        process_header_image(c, header_image_path)
    else:
        # Create a colored header if no image
        c.setFillColor(PRIMARY_COLOR)
        c.rect(0, height - header_height, width, header_height, fill=1)

    # Add title area
    c.setFillColor(LIGHT_BG_COLOR)
    c.rect(0, height - header_height - title_height, width, title_height, fill=1)

    c.setFillColor(PRIMARY_COLOR)
    c.setFont("Helvetica-Bold", 14)
    c.drawCentredString(width/2, height - header_height - title_height/2, title)

    # Add footer
    add_footer(c)

    # Calculate content area dimensions
    content_start_y = footer_height + margin
    content_end_y = height - header_height - title_height - margin
    content_height = content_end_y - content_start_y
    content_width = width - (2 * margin)

    # Return the canvas and dimensions for content placement
    return (c, content_start_y, content_width, content_height)